In [19]:
import pandas as pd
import numpy as np
import altair as alt
alt.data_transformers.enable('json')

from sublimpy import turbulence
import matplotlib.pyplot as plt
from sublimpy import tidy
import metpy.constants

# Open data

## Open precipitation dataset

In [20]:
precip_file = "~/Development/data/sublimationofsnow/tilden_precip_data/kettle_ponds_precip.csv"

precip_df = pd.read_csv(precip_file)

precip_df['date'] = pd.to_datetime(precip_df['date'])

acc_precip_on_first_day = precip_df.set_index('date').loc['20221130'].acc_prec

precip_df = precip_df.set_index('date').loc['20221130': '20230510'].reset_index()
precip_df['acc_prec']  = precip_df['acc_prec'] - acc_precip_on_first_day

## Open SOS Measurement Dataset

In [24]:
start_date = '20221130'
end_date = '20230509'
# open files
tidy_df = pd.read_parquet(f'tidy_df_{start_date}_{end_date}_noplanar_fit_clean.parquet')
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:end_date].reset_index()

In [ ]:
tidy_df_20221130_20230509_noplanar_fit_clean.parquet

In [6]:
# quick way to get variable info if we want it 
# import xarray as xr
# ds = xr.open_dataset("/storage/elilouis/sublimationofsnow/sosnoqc/isfs_20221228.nc")
# ds['SWE_p2_c']

# Calculate measured daily cumulative sublimation (for plotting)

In [26]:
tidy_df_daily = tidy_df[tidy_df.measurement == 'Cumulative sublimation measured'].set_index('time').groupby(
    [pd.Grouper(freq='1440Min'), 'tower', 'height', 'measurement', 'variable']
).max().reset_index()

# Plot

In [27]:
measurements_chart = alt.Chart(
    tidy_df_daily.query("measurement == 'Cumulative sublimation measured'").query("height > 1")
).mark_line(opacity=1, strokeWidth=1).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Cumulative sublimation (mm)").scale(domain=[-5, 60]),
    alt.Color("height:O").scale(scheme='turbo'),
    detail = 'variable:N',
    # tooltip = 'variable'
).properties(width = 250, height = 250)
measurements_chart

alt.Chart(...)

In [28]:

snowpillow_chart = alt.Chart(
    tidy_df.query("variable == 'SWE_p2_c'").dropna()
).transform_window(
    frame = [-48, 48],
    rolling_median = 'median(value)'
).mark_line().encode(
    alt.X("time:T").axis(labels=False).title(None),
    alt.Y("rolling_median:Q").title(["Snow water", "equivalent (mm)"])
).properties(width = 250, height = 83)

# snowpillow_and_precip_chart = snowpillow_chart +\
# alt.Chart(
#     precip_df
# ).mark_line(strokeDash=[2,4]).encode(
#     alt.X('date:T'),
#     alt.Y("acc_prec")
# )

snowpillow_chart

alt.Chart(...)

In [30]:
snowpillow_chart & measurements_chart

alt.VConcatChart(...)